
# Bayesian Optimization with Retrieval Optimizer

This notebook demonstrates how to use Bayesian optimization to tune Redis-based retrieval pipelines. Unlike a grid study—which tests all combinations—Bayesian optimization intelligently searches the configuration space, prioritizing promising settings based on previous results. This is especially useful when the number of possible configurations is large and exhaustive search would be too costly.

You'll define a study configuration, specify embedding models and search methods, and let the optimizer guide the search toward the best-performing retrieval setup.


# Installation

In [ ]:
%pip install redis-retrieval-optimizer

## Check version

In [1]:
import redis_retrieval_optimizer

redis_retrieval_optimizer.__version__

'0.4.1'

## Dataset

We'll import a dataset from the [beir benchmark IR project](https://github.com/beir-cellar/beir) to get going quickly. 

In [2]:
# Load data
from redis_retrieval_optimizer.corpus_processors import eval_beir

# check the link above for different datasets to try
beir_dataset_name = "nfcorpus"
data_folder = "data"

# Load sample data
corpus, queries, qrels = eval_beir.get_beir_dataset(beir_dataset_name)

/Users/robert.shelton/.pyenv/versions/3.11.9/lib/python3.11/site-packages/beir/util.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


15:01:54 beir.datasets.data_loader INFO   Loading Corpus...


  0%|          | 0/3633 [00:00<?, ?it/s]

15:01:54 beir.datasets.data_loader INFO   Loaded 3633 TEST Documents.
15:01:54 beir.datasets.data_loader INFO   Doc Example: {'text': 'Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (rang

Now that we have our data we will save it locally to the gitignored `data/` folder

In [3]:
import os

os.makedirs(data_folder, exist_ok=True)

In [4]:
import json

with open(f"data/{beir_dataset_name}_corpus.json", "w") as f:
    json.dump(corpus, f)

with open(f"data/{beir_dataset_name}_queries.json", "w") as f:
    json.dump(queries, f)

with open(f"data/{beir_dataset_name}_qrels.json", "w") as f:
    json.dump(qrels, f)

## Study config

In this directory there is a yaml file containing a configuration for a bayesian study that looks like this:

```yaml
# path to data files for easy read
corpus: "data/nfcorpus_corpus.json"
queries: "data/nfcorpus_queries.json"
qrels: "data/nfcorpus_qrels.json"

index_settings:
  name: "optimize"
  vector_field_name: "vector" # name of the vector field to search on
  text_field_name: "text" # name of the text field for lexical search
  from_existing: false
  vector_dim: 384 # should match first embedding model or from_existing
  additional_fields:
      - name: "title"
        type: "text"

optimization_settings:
  # defines the options optimization can take
  metric_weights:
    f1_at_k: 1
    embedding_latency: 1
    total_indexing_time: 1
  algorithms: ["hnsw"]
  vector_data_types: ["float16", "float32"]
  distance_metrics: ["cosine"]
  n_trials: 10
  n_jobs: 1
  ret_k: [1, 10] # potential range of value to be sampled during study
  ef_runtime: [10, 20, 30, 50]
  ef_construction: [100, 150, 200, 250, 300]
  m: [8, 16, 64]


search_methods: ["vector", "lin_combo"]
embedding_models:
  - type: "hf"
    model: "sentence-transformers/all-MiniLM-L6-v2"
    dim: 384
    embedding_cache_name: "vec-cache" # avoid names with including 'ret-opt' as this can cause collisions
    dtype: "float32"
```

## Running a study

To run a study simple pass the path to config, redis_url, and corpus processing function to the `run_bayes_study` function and the package will take care of the rest. 

In [ ]:
import os
from redis_retrieval_optimizer.bayes_study import run_bayes_study
from redis_retrieval_optimizer.corpus_processors import eval_beir
from dotenv import load_dotenv

# load environment variables containing necessary credentials
load_dotenv()

redis_url = os.environ.get("REDIS_URL", "redis://localhost:6379/0")

metrics = run_bayes_study(
    config_path="bayes_study_config.yaml",
    redis_url=redis_url,
    corpus_processor=eval_beir.process_corpus
)

In [6]:
metrics[["search_method", "algorithm", "vector_data_type", "ef_construction", "ef_runtime", "m", "avg_query_time", "total_indexing_time", "recall", "precision", "ndcg", "f1", "objective_value"]].sort_values(by="objective_value", ascending=False)

,search_method,algorithm,vector_data_type,ef_construction,ef_runtime,m,avg_query_time,total_indexing_time,recall,precision,ndcg,f1,objective_value
16,hybrid,hnsw,float32,250,30,16,0.001371,1292.178955,0.126754,0.284314,0.181060,0.122711,-0.569131
0,hybrid,hnsw,float16,250,30,64,0.001315,1303.661987,0.126754,0.284314,0.181060,0.122711,-0.569229
7,hybrid,hnsw,float32,300,30,16,0.001181,1496.173950,0.126754,0.284314,0.181060,0.122711,-0.569300
18,hybrid,hnsw,float16,250,30,16,0.001216,1271.833984,0.126754,0.284314,0.181060,0.122711,-0.569465
11,hybrid,hnsw,float32,300,30,16,0.001088,1498.623047,0.126754,0.284314,0.181060,0.122711,-0.569484
5,hybrid,hnsw,float16,300,50,64,0.001247,1166.741943,0.126754,0.284314,0.181060,0.122711,-0.569544
15,hybrid,hnsw,float32,250,10,8,0.001153,1292.178955,0.126754,0.284314,0.181060,0.122711,-0.569567
14,hybrid,hnsw,float16,300,30,16,0.001128,1290.228027,0.126754,0.284314,0.181060,0.122711,-0.569619
19,hybrid,hnsw,float32,250,50,8,0.001046,1415.468994,0.126754,0.284314,0.181060,0.122711,-0.569646
13,hybrid,hnsw,float32,150,30,16,0.001169,1199.935059,0.126754,0.284314,0.181060,0.122711,-0.569654
